### We will try to solve the basic problem of CartPole balancing using simple Q-learning.
[Article link](https://medium.com/analytics-vidhya/q-learning-is-the-most-basic-form-of-reinforcement-learning-which-doesnt-take-advantage-of-any-8944e02570c5)

In [11]:
import gym
import numpy as np
import matplotlib.pyplot as plt

In [2]:
state_space = 4  # number of state variables
action_space = 2  # number of actions


def Qtable(state_space, action_space, bin_size=30):
    bins = [np.linspace(-4.8, 4.8, bin_size),  # cart position
            np.linspace(-4, 4, bin_size),  # cart velocity
            np.linspace(-0.418, 0.418, bin_size),  # pole angle (radians)
            np.linspace(-4, 4, bin_size)]  # pole angular velocity

    q_table = np.random.uniform(low=-1, high=1, size=([bin_size] * state_space + [action_space]))
    return q_table, bins


def Discrete(state, bins):
    index = []
    for i in range(len(state)): index.append(np.digitize(state[i], bins[i]) - 1)
    return tuple(index)

In [12]:
def Q_learning(q_table, bins, episodes=5000, gamma=0.95, lr=0.1, timestep=100, epsilon=0.2):
    total_score = 0
    steps = 0
    curr_score_history = []
    for episode_i in range(1, episodes + 1):
        steps += 1
        # env.reset() => initial observation
        current_state = Discrete(env.reset(), bins)

        current_score = 0
        done = False
        while not done:
            if episode_i % timestep == 0: env.render()
            if np.random.uniform(0, 1) < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[current_state])
            observation, reward, done, info = env.step(action)
            next_state = Discrete(observation, bins)
            current_score += reward  # Accumulate the current_score until done

            if not done:
                max_future_q = np.max(q_table[next_state])
                current_q = q_table[current_state + (action,)]
                new_q = (1 - lr) * current_q + lr * (reward + gamma * max_future_q)
                q_table[current_state + (action,)] = new_q
            else:
                total_score += current_score
            current_state = next_state
        curr_score_history.append(current_score)
        avg_score = total_score / episode_i
        if episode_i % timestep == 0: print('Average score after {} episodes:- {}'.format(episode_i, avg_score))
        if avg_score >= 150 or current_score >= 150:  # This success score is completely upto us.
            print('Problem solved in episode {} with steps {}'.format(episode_i, steps))
            return curr_score_history
    return curr_score_history

Looks like using current score as stopping criterion is not enough, since there's wild fluctuations in the performance of the agent. It is indeed better to use Average score to achieve the desired score, although this doesn't guarantee anything if run again with the same Q-table.

In [ ]:
env = gym.make('CartPole-v1')
q_table, bins = Qtable(4, 2)
score_history = Q_learning(q_table, bins, episodes=10000, gamma=0.995, lr=0.15)
env.close()
# plt.plot(score_history)
# plt.show()